To test if and how the model functions in extreme conditions, the model is run with the 500yr floodmap for 100 iterations. This is the most extreme floodmap that was given in the base model. It displays a severe flood that only happens every 500 years. In the model, this will result in high values for flood depth of households. The following hypotheses are stated on how the model will function in these conditions to show that the model still functions as it is supposed to:

1. The total damage actually experienced will be a lot higher as adaptation measures are not up to the high values of flood depth
2. A lot more households will opt for adaptation measures since the estimated prevented damage will exceed the adaptation costs more often
3. Another reason more households will opt for adaptation measures is because the location of households will more likely be in a floodplain, resulting in a higher chance of receiving an initial flood perception of 3 or 4, which is one of the main factors for choosing adaptation measures

In [1]:
from model import AdaptationModel
from mesa.batchrunner import batch_run
import pandas as pd

In [2]:
#Extreme values test
model_params = {
    "number_of_households": 1000,
    "number_of_steps": 80,
    "flood_map_choice": ["harvey", "100yr", "500yr"],
    "network": "watts_strogatz"
}

results = batch_run(
    model_cls=AdaptationModel,
    parameters=model_params,
    number_processes=4,  # Set to None if you want to use all CPUs
    iterations=100,  # Number of iterations for each parameter combination
    data_collection_period=-1,  # Collect data at the end of each run
    max_steps=model_params["number_of_steps"],  # Maximum number of model steps
    display_progress=True  # Display progress bar
)

df = pd.DataFrame(results)

model_vars_extreme_values_df = df

  0%|          | 0/300 [00:00<?, ?it/s]

In [3]:
def unpack_dictionary(row, column_name):
    return pd.Series(row[column_name])

#base_case_unpacked = base_case_100_iterations.apply(unpack_dictionary['AverageDamagePerIncomeLabel'], axis = 1)
first_column_unpacked = model_vars_extreme_values_df.apply(unpack_dictionary, args=('AverageDamagePerIncomeLabel',), axis=1)
second_column_unpacked = model_vars_extreme_values_df.apply(unpack_dictionary, args=('AverageIncomeToDamageRatio',), axis=1)
extreme_values_unpacked = pd.concat([model_vars_extreme_values_df, first_column_unpacked, second_column_unpacked], axis=1)
model_vars_extreme_values_clean = extreme_values_unpacked.drop(['AverageDamagePerIncomeLabel', 'AverageIncomeToDamageRatio'], axis=1)
model_vars_extreme_values_clean

,RunId,iteration,Step,number_of_households,number_of_steps,flood_map_choice,network,TotalAdaptedHouseholds,TotalActualDamage,TotalExpectedDamage,TotalAdaptationCosts,TotalCostsOfSubsidies,AverageDamagePerPoorHousehold,AverageDamagePerMiddleClassHousehold,AverageDamagePerRichHousehold,AverageIncomeToDamagePoorHousehold,AverageIncomeToDamageMiddleClassHousehold,AverageIncomeToDamageRichHousehold
0,0,0,14,1000,80,harvey,watts_strogatz,544,4.709781e+07,4.972030e+07,2075365,0,22715.367157,48211.893268,104519.743342,4.612960,1.644535,1.199793
1,2,0,31,1000,80,500yr,watts_strogatz,417,2.222798e+07,2.379461e+07,1844925,0,11900.904269,20983.909927,48968.959331,2.351506,0.721397,0.543312
2,1,0,48,1000,80,100yr,watts_strogatz,317,1.372802e+07,1.480638e+07,1600920,0,6638.366233,14334.351130,26910.397905,1.407224,0.476962,0.308374
3,5,1,16,1000,80,500yr,watts_strogatz,384,2.010436e+07,2.154321e+07,1793980,0,7851.177012,20469.100865,49965.909125,1.572970,0.680746,0.575159
4,3,1,53,1000,80,harvey,watts_strogatz,589,4.777509e+07,5.050010e+07,4701615,0,26022.510915,45162.400866,109430.351939,5.342378,1.561452,1.254880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,98,55,1000,80,500yr,watts_strogatz,357,2.137778e+07,2.265748e+07,2151075,0,9590.297553,22575.464957,43941.022307,1.915211,0.768634,0.503647
296,297,99,16,1000,80,harvey,watts_strogatz,586,4.297254e+07,4.600827e+07,2721615,0,22367.835930,42564.368419,106320.839645,4.527344,1.448382,1.204534
297,295,98,69,1000,80,100yr,watts_strogatz,314,1.372589e+07,1.465051e+07,2119400,0,7946.539588,13327.761127,31548.537921,1.536154,0.461211,0.360239
298,298,99,42,1000,80,100yr,watts_strogatz,276,1.412606e+07,1.517185e+07,1537025,0,7588.228615,13499.524720,34157.269477,1.450464,0.454982,0.400971


In [4]:
output_data_path = r'../output_data/ExtremeValuesModelVars.csv'
model_vars_extreme_values_clean.to_csv(output_data_path, index=False)